In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from sklearn import linear_model
from scipy.stats.stats import pearsonr

import os
import sys
import re
import pdb
import glob

sys.path.insert(0, '/Users/jkinney/github/anylogo/')
#sys.path.append('/Users/jkinney/github/anylogo')
sys.path.append('../src')

import utils
from helper_functions import gelx, gely
from anylogo import logomaker

# Set other things
%matplotlib inline
plt.ion()
sns.set_style("whitegrid", {'axes.grid' : False})

In [9]:
# Set splicing threshold
splicing_threshod = 0

# Set consensus sequence
cons_seq = 'CAGGUAAGU'

# Set dataset
dataset_name = 'brca2_9nt'

In [10]:
# Load ratios
in_file = '../output/ratios_9nt_ss_locus.txt'
df = pd.read_csv(in_file,sep='\t')
df.set_index('ss',inplace=True,drop=True)
site_length = len(df.index[0])    

In [11]:
# Normalize each column by consens sequence value
for col in df.columns:
    df.loc[:,col] = 100*df.loc[:,col]/df.loc[cons_seq,col]

In [12]:
# Set measurement data frame
measurements = df[dataset_name]

# Drop nan rows
measurements = measurements.dropna()

# Keep only GU splice sites
indices = [ss[3:5]=='GU' for ss in measurements.index]
measurements = measurements[indices]

# Filter for substantial values
measurements = measurements[measurements >= splicing_threshod]
len(measurements)

16216

In [13]:
# Compute feature vectors for single positions
bases = 'ACGU'
variable_positions = np.array([0,1,2,5,6,7,8])

features_mat_df = pd.DataFrame(index=df.index)
for i in variable_positions:
    for b in bases:
        col_name = '%d%s'%(i,b)
        features_mat_df.loc[:,col_name] = [float(ss[i]==b) for ss in df.index]

In [14]:
# Compute feature vector for all pairs of positions
features_pair_df = pd.DataFrame(index=features_mat_df.index)
for i in variable_positions:
    for j in variable_positions:
        if j > i:
            for b in bases:
                for c in bases:
                    pos1_col = '%d%s'%(i,b)
                    pos2_col = '%d%s'%(j,c)
                    pair_col = '%s_%s'%(pos1_col,pos2_col)
                    features_pair_df.loc[:,pair_col] = features_mat_df.loc[:,pos1_col]*features_mat_df.loc[:,pos2_col]
features_pair_df.head()

,0A_1A,0A_1C,0A_1G,0A_1U,0C_1A,0C_1C,0C_1G,0C_1U,0G_1A,0G_1C,...,7C_8G,7C_8U,7G_8A,7G_8C,7G_8G,7G_8U,7U_8A,7U_8C,7U_8G,7U_8U
ss,,,,,,,,,,,,,,,,,,,,,
AAAGCAAAA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAAGCAAAC,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAAGCAAAG,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAAGCAAAU,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAAGCAACA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Concatinate all features into one data frame
features_all_df = features_mat_df.merge(features_pair_df,left_index=True,right_index=True)
features_all_df.head()

,0A,0C,0G,0U,1A,1C,1G,1U,2A,2C,...,7C_8G,7C_8U,7G_8A,7G_8C,7G_8G,7G_8U,7U_8A,7U_8C,7U_8G,7U_8U
ss,,,,,,,,,,,,,,,,,,,,,
AAAGCAAAA,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAAGCAAAC,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAAGCAAAG,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAAGCAAAU,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAAGCAACA,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Compute design matrix 
X_all = features_all_df.loc[measurements.index,:]
X_all.shape

(16216, 364)

In [22]:
# Do logistic regression fit
reg_all = linear_model.LogisticRegression ()
reg_all.fit (X=X_all, y=measurements)

ValueError: Unknown label type: 'continuous'

In [20]:


x = measurements
y = reg_all.predict(X_all)
plt.plot(x,y,'.')
r,p = pearsonr(x,y)

plt.xlabel('measurements')
plt.ylabel('logistic model predictions')
plt.title('Logistic regression, $R^2 = %0.2f$'%r**2)

ValueError: Unknown label type: 'continuous'

In [ ]:
# Comptue residuals
residuals = measurements - reg_mat.predict(X_mat)

In [ ]:
# Compute design matrix 
X_pair = features_pair_df.loc[residuals.index,:]
X_pair.shape

In [ ]:
# Do linear regression with small rige penalty to fix gauge
reg_pair = linear_model.Ridge (alpha = .0001)
reg_pair.fit(X_pair, residuals)

x = measurements
y = reg_mat.predict(X_mat) + reg_pair.predict(X_pair)
plt.plot(x,y,'.')
r,p = pearsonr(x,y)

plt.xlabel('measurements')
plt.ylabel('matrix + pairwise predictions')
plt.title('Matrix regression, $R^2 = %0.2f$'%r**2)


In [ ]:
# Plot coefficients
effect_vec = pd.DataFrame(index=features_pair_df.columns)
effect_vec['value'] = reg_pair.coef_
all_cols = ['%d%s'%(i,b) for i in range(0,9) for b in bases]

effect_mat = pd.DataFrame(index=all_cols, columns=all_cols).fillna(0)
for pair in effect_vec.index:
    pos1, pos2 = pair.split('_')
    effect_mat.loc[pos1,pos2] = effect_vec.loc[pair,'value']
    effect_mat.loc[pos2,pos1] = effect_vec.loc[pair,'value']
effect_mat.head()

In [ ]:
# Create annotation df
annotation_df = pd.DataFrame(columns=['pos','base'],
                             data=[[str(i),b] for i in [-3,-2,-1,1,2,3,4,5,6] for b in bases])


# Create figure and axes
fig, ax = plt.subplots(figsize=[10,8])

# Plot heatmap
sns.heatmap(effect_mat.iloc[::-1,:])

# Plot gridlines
for i in range(len(effect_mat)+1):
    if i%4==0:
        ax.axhline(i, color='gray', linewidth=1)
        ax.axvline(i, color='gray', linewidth=1)
        
# Annotate x- and y-axes
gelx(ax, annotation_df, annotation_spacing=1.5, fontsize=10)
gely(ax, annotation_df.iloc[::-1,:], annotation_spacing=1.5, fontsize=10)

# Draw diagonal line
xs = ax.get_xlim()
ys = xs
ax.plot(xs, ys, linewidth=2, color='gray')

from matplotlib.patches import Polygon

# Draw masking triangle
a = xs[0]
b = xs[1]
xy = np.array([
        [a, a],
        [b, a],
        [b, b]])
triangle = Polygon(xy, closed=True, facecolor='white', zorder=10)
ax.add_patch(triangle)

# Save figure
fig.savefig(dataset_name + '_pairwise.pdf')